# LangChain 核心模块 Agent 

有些应用程序不仅需要预先确定的LLM/其他工具调用链，而且可能需要根据用户输入的不同而产生不同的链条。

在这些类型的链条中，有一个“代理人”可以访问一套工具。

根据用户输入，代理人可以决定是否调用其中任何一个工具。

在本文档的本节中，我们首先从“快速入门”开始，介绍如何以端到端的方式使用与代理人相关的所有内容。

## 1.OPENAI Function

In [1]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

In [2]:
from langchain.chat_models import ChatOpenAI

# 使用 GPT-3.5-turbo
llm = ChatOpenAI(temperature=0)

d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [3]:

from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.schema import SystemMessage
from langchain.agents import OpenAIFunctionsAgent

system_message = SystemMessage(content="你是非常强大的AI助手，但在计算单词长度方面不擅长。")
prompt = OpenAIFunctionsAgent.create_prompt(system_message=system_message)

In [5]:
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)


d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.openai_functions_agent.base.OpenAIFunctionsAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_openai_functions_agent instead.
  warn_deprecated(


In [6]:
from langchain.agents import AgentExecutor

# 实例化 OpenAIFunctionsAgent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [7]:
agent_executor.run("单词“educa”中有多少个字母?")

d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5单词“educa”中有5个字母。

> Finished chain.


'单词“educa”中有5个字母。'

## 2.ReAct
ReAct 核心思想是 推理+操作，本示例以 Google Search 和 LLM Math 作为可选操作集合（toolkits），实现 ReAct 功能。

In [8]:
import os
import socket
import socks

socks.set_default_proxy(socks.SOCKS5, "127.0.0.1", 10808)
socket.socket = socks.socksocket

os.environ['SERPAPI_API_KEY'] = 'fbf6cd606169327528551dab839c53f8a698da82e5229b39118312b276d81559'


In [9]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)


d:\sdk\Anaconda3\envs\bigmodel\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [10]:

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType

#加载 LangChain 内置的 Tools
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [13]:
# 实例化 ZERO_SHOT_REACT Agent
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 Agent    
agent.run("2024年中国邮政集团有限公司新任的总经理是谁？")




> Entering new AgentExecutor chain...
 I should search for the answer using a search engine.
Action: Search
Action Input: "2024年中国邮政集团有限公司新任总经理"
Observation: 据了解，3月31日，中国邮政集团有限公司召开领导班子（扩大）会议。 受中央组织部领导委托，中央组织部有关干部局负责同志宣布了中央关于中国邮政集团有限公司总经理任职的决定：郑国雨同志任中国邮政集团有限公司董事、总经理、党组副书记，免去其中国工商银行执行董事、副行长、党委委员职务。
Thought: I now know the final answer.
Final Answer: 郑国雨

> Finished chain.


'郑国雨'

## 3.Self ask with search

In [14]:

from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0)

In [15]:
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [16]:

# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [19]:

# 实际运行 Agent，查询问题（正确）
self_ask_with_search.run(
    "What is the hometown of the reigning men's U.S. Open champion?"
)



> Entering new AgentExecutor chain...
Could not parse output:  No. (Translation: Where is the hometown of the defending men's singles champion of the US Open?)
Intermediate answer: Invalid or incomplete response
So the final answer is: Invalid or incomplete response

> Finished chain.


'Invalid or incomplete response'